In [ ]:
import pandas as pd
import numpy as np
from pandas_datareader import data, wb
import datetime

import sys
sys.path.append('../..')
from backtester import matlab, backtester
from backtester.analysis import *
from pandas.tseries.offsets import *

In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
#
# random time series
#
date = pd.date_range(start='2015-01', end='2015-05', freq='D')
price = np.random.randint(-100, 100, size=len(date))
df = pd.DataFrame(price, index=date, columns=['close'])
df = df.cumsum()

In [ ]:
#
# Real ticker data
#
start = datetime.datetime(2010, 1, 1)
end = datetime.datetime.now()

price_data = data.DataReader('SPY', 'yahoo', start, end)
adjclose_series = price_data['Adj Close']
adjclose_series.name = 'close'

df = pd.DataFrame(adjclose_series)

In [ ]:
strategyname = 'strategy_180113'
d, info = matlab.loaddata('../../mat/'+strategyname+'.mat')

df = pd.DataFrame()
df['close'] = d.exo

In [ ]:
figsize(5,5)
d.exo.plot()

In [ ]:
info

## Renko calculation

## High/Low of renko bricks is only needed for visual representation

In [ ]:
box_size = 100

box_start = 0
box_start_idx = None

box_end = 0
box_quantity = 0

temp_l = []

# Simple renko algorihtm


# 
for i in range(len(df)):
    if box_start == 0:
        box_start = df.close[i]

    else:
        box_start = box_start
        price_move = df.close[i] - box_start
        
        # First of all we need to set box size. 
        # Then calculate price movement. 
        # If price movement is more or equal than box size - renko bar(or bars) will be added
        
        if np.abs(price_move) >= box_size:
            
            # After we calculate box_quantity(price move divided by box size)
            # This number defines how much renko bars will be registred
            box_quantity = np.int32(np.floor(np.abs(price_move / box_size)))
            box_index = df.close.index[i]

            for b in range(int(box_quantity)):
            # Let say, index is 2015-01-01, box_start = 100, box_quantity = 3, box size = 10, price move > 0
            # So renko bar 1 will have next parameters - 
            # 1)index 2015-01-01
            # 2)open = 100
            # 3)close = 110(box_start + box_size)
            # 4)type = up
            
            # Next renko bar will have next parameters -
            # 1)index 2015-01-01
            # 2)open = 110(previous renko bar close)
            # 3)close = 120(open + box_size)
            # 4)type = up
            
            # And so on..
            
            # After all we adding renko bars dict to list and convert it to DF
            
                if price_move > 0:
                    if box_end == 0:
                        d = {'date': box_index, 'open': box_start, 'close': box_start + box_size, 'type': 'up'}
                        box_end = d['close']
                        temp_l.append(d)
                        
                    else:
                        d = {'date': box_index, 'open': box_end, 'close': box_end + box_size,
                            'type': 'up'}
                        
                        box_end = d['close']
                        temp_l.append(d)

                if price_move < 0:
                    if box_end == 0:
                        d = {'date': box_index, 'open': box_start, 'close': box_start - box_size, 'type': 'down'}
                        box_end = d['close']
                        temp_l.append(d)
                        
                    else:           
                        d = {'date': box_index, 'open': box_end, 'close': box_end - box_size, 
                             'type': 'down'}
                        
                        box_end = d['close']
                        temp_l.append(d)
                        
            box_start = df.close[i]
            
renko_df = pd.DataFrame(temp_l)

del temp_l

high_l = []
low_l = []
for i in range(len(renko_df)):
    if renko_df.close[i] > renko_df.open[i]:
        high_l.append(renko_df.close[i])
        low_l.append(renko_df.open[i])
        
    if renko_df.close[i] < renko_df.open[i]:
        high_l.append(renko_df.open[i])
        low_l.append(renko_df.close[i])
        
renko_df['low'] = low_l
renko_df['high'] = high_l

del low_l
del high_l

## Defining peaks and falls and their prices

In [ ]:
# Peaks
renko_peak = ((renko_df.type == 'down') & (renko_df.type.shift(1) == 'down') 
          & (renko_df.type.shift(2) == 'up') & (renko_df.type.shift(3) == 'up'))

renko_df['peak'] = renko_peak

# Adding underlying_pick_price column
ul_peak_price = df.close[df.close.index.isin(renko_df[renko_df.peak.shift(-2) == True].date)]
ul_peak_price.name = 'underlying_peak_price'

renko_df = renko_df.join(ul_peak_price, on=['date'])
renko_df.underlying_peak_price = renko_df.underlying_peak_price.shift(2) #This needed for avoiding future reference problem

renko_df.loc[renko_df.peak == False, 'underlying_peak_price'] = np.NaN # Drop all values which available earlier than peak/fall occurs

renko_df['renko_peak_price'] = renko_df.close[renko_df.peak.shift(-2) == True]
renko_df.renko_peak_price = renko_df.renko_peak_price.shift(2) #This needed for avoiding future reference problem

In [ ]:
# Falls
renko_fall = ((renko_df.type == 'up') & (renko_df.type.shift(1) == 'up') 
          & (renko_df.type.shift(2) == 'down') & (renko_df.type.shift(3) == 'down'))

renko_df['fall'] = renko_fall

# Adding underlying_fall_price column 
ul_fall_price = df.close[df.close.index.isin(renko_df[renko_df.fall.shift(-2) == True].date)]
ul_fall_price.name = 'underlying_fall_price'

renko_df = renko_df.join(ul_fall_price, on=['date'])
renko_df.underlying_fall_price = renko_df.underlying_fall_price.shift(2) #This needed for avoiding future reference problem

renko_df.loc[renko_df.fall == False, 'underlying_fall_price'] = np.NaN # Drop all values which available earlier than peak/fall occurs

renko_df['renko_fall_price'] = renko_df.close[renko_df.fall.shift(-2) == True]
renko_df.renko_fall_price = renko_df.renko_fall_price.shift(2) #This needed for avoiding future reference problem

In [ ]:
renko_df.renko_peak_price = renko_df.renko_peak_price.fillna(method='ffill')
renko_df.renko_fall_price = renko_df.renko_fall_price.fillna(method='ffill')

## Flat and trend patterns

In [ ]:
renko_flat = (((renko_df.type == 'up') & (renko_df.type.shift(1) == 'down') 
                & (renko_df.type.shift(2) == 'up') & (renko_df.type.shift(3) == 'down')) | 
              ((renko_df.type == 'down') & (renko_df.type.shift(1) == 'up') & (renko_df.type.shift(2) == 'down') 
             & (renko_df.type.shift(3) == 'up')))

renko_df['flat'] = renko_flat

renko_trend_up = ((renko_df.type == 'up') & (renko_df.type.shift(1) == 'up') & (renko_df.type.shift(2) == 'up'))
renko_trend_down = ((renko_df.type == 'down') & (renko_df.type.shift(1) == 'down') & (renko_df.type.shift(2) == 'down'))

renko_df['trend_up'] = renko_trend_up
renko_df['trend_down'] = renko_trend_down

## Small double top/bottom pattern

In [ ]:
renko_small_double_top = ((renko_df.type == 'down') & (renko_df.type.shift(1) == 'down') & (renko_df.type.shift(2) == 'up')
                         & (renko_df.type.shift(3) == 'down') & (renko_df.type.shift(4) == 'up') & (renko_df.type.shift(5) == 'up'))

renko_df['small_double_top'] = renko_small_double_top

renko_small_double_bottom = ((renko_df.type == 'up') & (renko_df.type.shift(1) == 'up') & (renko_df.type.shift(2) == 'down')
                         & (renko_df.type.shift(3) == 'up') & (renko_df.type.shift(4) == 'down') & (renko_df.type.shift(5) == 'down'))

renko_df['small_double_bottom'] = renko_small_double_bottom

## Trend correction pattern( no.7 in renko patterns.pdf)

In [ ]:
renko_up_trend_correction = ((renko_df.type == 'up') & (renko_df.type.shift(1) == 'up') & (renko_df.type.shift(2) == 'down')
                            & (renko_df.type.shift(3) == 'up') & (renko_df.type.shift(4) == 'up'))

renko_df['up_trend_correction'] = renko_up_trend_correction

renko_down_trend_correction = ((renko_df.type == 'down') & (renko_df.type.shift(1) == 'down') & (renko_df.type.shift(2) == 'up')
                            & (renko_df.type.shift(3) == 'down') & (renko_df.type.shift(4) == 'down'))

renko_df['down_trend_correction'] = renko_down_trend_correction

# Peak/fall prices combinations

### Defining peak/fall price direction relative to previous peak/fall price

In [ ]:
renko_peak_df = renko_df[renko_df.peak == True]

renko_peak_price_move = np.array([None] * len(renko_peak_df.type))
#renko_peak_price_move = np.empty_like(renko_peak_df.type) # This code crashes the python...

for i in range(len(renko_peak_df)):
    if i > 0:
        if renko_peak_df.renko_peak_price.values[i] > renko_peak_df.renko_peak_price.values[i-1]: 
            renko_peak_price_move[i] = 'up'
            
        elif renko_peak_df.renko_peak_price.values[i] < renko_peak_df.renko_peak_price.values[i-1]:
            renko_peak_price_move[i] = 'down'
            
        elif renko_peak_df.renko_peak_price.values[i-1] == renko_peak_df.renko_peak_price.values[i]:
            renko_peak_price_move[i] = 'same'

renko_df = renko_df.join(pd.Series(renko_peak_price_move, index=renko_peak_df.index, name='renko_peak_price_move').replace([None],np.NaN))

del renko_peak_df

In [ ]:
renko_fall_df = renko_df[renko_df.fall == True]

renko_fall_price_move = np.array([None] * len(renko_fall_df.type))

for i in range(len(renko_fall_df)):
    if i > 0:
        if renko_fall_df.renko_fall_price.values[i] > renko_fall_df.renko_fall_price.values[i-1]: 
            renko_fall_price_move[i] = 'up'
            
        elif renko_fall_df.renko_fall_price.values[i] < renko_fall_df.renko_fall_price.values[i-1]:
            renko_fall_price_move[i] = 'down'
            
        elif renko_fall_df.renko_fall_price.values[i-1] == renko_fall_df.renko_fall_price.values[i]:
            renko_fall_price_move[i] = 'same'

renko_df = renko_df.join(pd.Series(renko_fall_price_move, index=renko_fall_df.index, name='renko_fall_price_move').replace([None],np.NaN))

del renko_fall_df

## Patterns
#### Peak
#### Consecutive peak price movements count

In [ ]:
renko_peak_price_move_ser = renko_df.renko_peak_price_move.dropna()
renko_peak_price_move_ser_prev = renko_df.renko_peak_price_move.dropna().shift(1)

up_move_count = []
up_move_counter = 0

down_move_count = []
down_move_counter = 0

same_move_count = []
same_move_counter = 0

for i in renko_peak_price_move_ser.index:
    
    if i > 0:
    # Consec Up peak price movements
        if (renko_peak_price_move_ser[i] == 'up') & (renko_peak_price_move_ser_prev[i] != 'up'):
            up_move_counter = 1
            up_move_count.append(up_move_counter)
            
        elif (renko_peak_price_move_ser[i] == 'up') & (renko_peak_price_move_ser_prev[i] == 'up'):
            up_move_counter += 1
            up_move_count.append(up_move_counter)
            
        elif (renko_peak_price_move_ser[i] != 'up') & (renko_peak_price_move_ser_prev[i] == 'up'):
            up_move_counter = 0
            up_move_count.append(up_move_counter)
            
        elif (renko_peak_price_move_ser[i] != 'up') & (renko_peak_price_move_ser_prev[i] != 'up'):
            up_move_counter = 0
            up_move_count.append(up_move_counter)
            
    # Consec down peak price movements  
        if (renko_peak_price_move_ser[i] == 'down') & (renko_peak_price_move_ser_prev[i] != 'down'):
            down_move_counter = 1
            down_move_count.append(down_move_counter)
            
        elif (renko_peak_price_move_ser[i] == 'down') & (renko_peak_price_move_ser_prev[i] == 'down'):
            down_move_counter += 1
            down_move_count.append(down_move_counter)
            
        elif (renko_peak_price_move_ser[i] != 'down') & (renko_peak_price_move_ser_prev[i] == 'down'):
            down_move_counter = 0
            down_move_count.append(down_move_counter)
            
        elif (renko_peak_price_move_ser[i] != 'down') & (renko_peak_price_move_ser_prev[i] != 'down'):
            down_move_counter = 0
            down_move_count.append(down_move_counter)
            
    # Consec same peak price movements
        if (renko_peak_price_move_ser[i] == 'same') & (renko_peak_price_move_ser_prev[i] != 'same'):
            same_move_counter = 1
            same_move_count.append(same_move_counter)
            
        elif (renko_peak_price_move_ser[i] == 'same') & (renko_peak_price_move_ser_prev[i] == 'same'):
            same_move_counter += 1
            same_move_count.append(same_move_counter)
            
        elif (renko_peak_price_move_ser[i] != 'same') & (renko_peak_price_move_ser_prev[i] == 'same'):
            same_move_counter = 0
            same_move_count.append(same_move_counter)
            
        elif (renko_peak_price_move_ser[i] != 'same') & (renko_peak_price_move_ser_prev[i] != 'same'):
            same_move_counter = 0
            same_move_count.append(same_move_counter)

renko_df['renko_peak_price_up_move_count'] = pd.Series(up_move_count, index=renko_peak_price_move_ser.index, 
                                                       name='renko_peak_price_up_move_count')
  
renko_df['renko_peak_price_down_move_count'] = pd.Series(down_move_count, index=renko_peak_price_move_ser.index, 
                                                         name='renko_peak_price_down_move_count')

renko_df['renko_peak_price_same_move_count'] = pd.Series(same_move_count, index=renko_peak_price_move_ser.index, 
                                                         name='renko_peak_price_same_move_count')

#### Fall
#### Consecutive peak price movements count

In [ ]:
renko_fall_price_move_ser = renko_df.renko_fall_price_move.dropna()
renko_fall_price_move_ser_prev = renko_df.renko_fall_price_move.dropna().shift(1)

up_move_count = []
up_move_counter = 0

down_move_count = []
down_move_counter = 0

same_move_count = []
same_move_counter = 0

for i in renko_fall_price_move_ser.index:
    
    if i > 0:
    # Consec Up fall price movements
        if (renko_fall_price_move_ser[i] == 'up') & (renko_fall_price_move_ser_prev[i] != 'up'):
            up_move_counter = 1
            up_move_count.append(up_move_counter)
            
        elif (renko_fall_price_move_ser[i] == 'up') & (renko_fall_price_move_ser_prev[i] == 'up'):
            up_move_counter += 1
            up_move_count.append(up_move_counter)
            
        elif (renko_fall_price_move_ser[i] != 'up') & (renko_fall_price_move_ser_prev[i] == 'up'):
            up_move_counter = 0
            up_move_count.append(up_move_counter)
            
        elif (renko_fall_price_move_ser[i] != 'up') & (renko_fall_price_move_ser_prev[i] != 'up'):
            up_move_counter = 0
            up_move_count.append(up_move_counter)
            
    # Consec down fall price movements  
        if (renko_fall_price_move_ser[i] == 'down') & (renko_fall_price_move_ser_prev[i] != 'down'):
            down_move_counter = 1
            down_move_count.append(down_move_counter)
            
        elif (renko_fall_price_move_ser[i] == 'down') & (renko_fall_price_move_ser_prev[i] == 'down'):
            down_move_counter += 1
            down_move_count.append(down_move_counter)
            
        elif (renko_fall_price_move_ser[i] != 'down') & (renko_fall_price_move_ser_prev[i] == 'down'):
            down_move_counter = 0
            down_move_count.append(down_move_counter)
            
        elif (renko_fall_price_move_ser[i] != 'down') & (renko_fall_price_move_ser_prev[i] != 'down'):
            down_move_counter = 0
            down_move_count.append(down_move_counter)
            
    # Consec same fall price movements
        if (renko_fall_price_move_ser[i] == 'same') & (renko_fall_price_move_ser_prev[i] != 'same'):
            same_move_counter = 1
            same_move_count.append(same_move_counter)
            
        elif (renko_fall_price_move_ser[i] == 'same') & (renko_fall_price_move_ser_prev[i] == 'same'):
            same_move_counter += 1
            same_move_count.append(same_move_counter)
            
        elif (renko_fall_price_move_ser[i] != 'same') & (renko_fall_price_move_ser_prev[i] == 'same'):
            same_move_counter = 0
            same_move_count.append(same_move_counter)
            
        elif (renko_fall_price_move_ser[i] != 'same') & (renko_fall_price_move_ser_prev[i] != 'same'):
            same_move_counter = 0
            same_move_count.append(same_move_counter)

renko_df['renko_fall_price_up_move_count'] = pd.Series(up_move_count, index=renko_fall_price_move_ser.index, 
                                                       name='renko_fall_price_up_move_count')
  
renko_df['renko_fall_price_down_move_count'] = pd.Series(down_move_count, index=renko_fall_price_move_ser.index, 
                                                         name='renko_fall_price_down_move_count')

renko_df['renko_fall_price_same_move_count'] = pd.Series(same_move_count, index=renko_fall_price_move_ser.index, 
                                                         name='renko_fall_price_same_move_count')

In [ ]:
renko_df

In [ ]:
renko_df.renko_fall_price_down_move_count.dropna()

## Consecutive up/down brick count

In [ ]:
up_count = np.zeros_like(renko_df.index)
up_counter = 0

down_count = np.zeros_like(renko_df.index)
down_counter = 0


for i in range(len(renko_df.index)):
    if i > 0:
    
        if (renko_df.type[i] == 'up') & (renko_df.type[i-1] == 'down'):
            up_counter = 1
            up_count[i] = up_counter

        elif (renko_df.type[i] == 'up') & (renko_df.type[i-1] == 'up'):
            up_counter += 1
            up_count[i] = up_counter

        elif (renko_df.type[i] == 'down') & (renko_df.type[i-1] == 'up'):
            up_counter = 0
            up_count[i] = up_counter
            
        else:
            up_counter = 0
            up_count[i] = up_counter
        
        
        
        if (renko_df.type[i] == 'down') & (renko_df.type[i-1] == 'up'):
            down_counter = 1
            down_count[i] = down_counter

        elif (renko_df.type[i] == 'down') & (renko_df.type[i-1] == 'down'):
            down_counter += 1
            down_count[i] = down_counter

        elif (renko_df.type[i] == 'up') & (renko_df.type[i-1] == 'down'):
            down_counter = 0
            down_count[i] = down_counter
            
        else:
            down_counter = 0
            down_count[i] = down_counter
            
renko_df['up_count'] = up_count
renko_df['down_count'] = down_count

In [ ]:
renko_df[renko_df.up_count != 0].groupby('up_count').count().div(len(renko_df[renko_df.up_count != 0])).close

In [ ]:
renko_df[renko_df.down_count != 0].groupby('down_count').count().div(len(renko_df[renko_df.down_count != 0])).close

In [ ]:
import plotly.plotly as py
from plotly.tools import FigureFactory as FF
from datetime import datetime
import plotly.graph_objs as go

py.sign_in('dmveden', 'rzbs7xw8ft')
#py.sign_in('tmqr', '8rctfyu197')

fig = FF.create_candlestick(renko_df.open, renko_df.high, renko_df.low, renko_df.close, dates=renko_df.index)
py.iplot(fig)

In [ ]:
trace = go.Scatter(
    x = df.index,
    y = df.close
)

data = [trace]

# Plot and embed in ipython notebook!
py.iplot(data, filename='basic-line')

In [ ]:
figsize(20,10)
renko_df.close[:].plot()
#renko_df.low[:].plot()

(renko_df.renko_fall_price_up_move_count==2).plot(secondary_y=True)

#(renko_df.renko_fall_price_up_move_count==2).plot(secondary_y=True)

#renko_df.small_double_bottom.plot(secondary_y=True)
#renko_df.small_double_top.plot(secondary_y=True)

In [ ]:
strategyname = 'strategy_180113'
d, info = matlab.loaddata('../../mat/'+strategyname+'.mat')

In [ ]:
df = pd.DataFrame()
df['close'] = d.exo

In [ ]:
#renko_df.index = renko_df.date

df = df.join(renko_df.set_index('date')[['peak','fall','flat','trend_up','trend_down','small_double_bottom',
                                        'small_double_top','up_trend_correction', 'down_trend_correction']])
df = df.fillna(False)

df = df.join(renko_df.set_index('date')[['renko_peak_price_move','renko_fall_price_move']])
df = df.join(renko_df.set_index('date')[['up_count', 'down_count', 'renko_peak_price_up_move_count',
                                           'renko_peak_price_down_move_count', 'renko_peak_price_same_move_count',
                                           'renko_fall_price_up_move_count', 'renko_fall_price_down_move_count',
                                           'renko_fall_price_same_move_count']])
#df = df.join(renko_df.fall, how='right')
#df = df.join(renko_df.flat, how='right')
#df = df.join(renko_df.trend_up)
#df = df.join(renko_df.trend_down)
#df = df.join(renko_df.small_double_bottom)
#df = df.join(renko_df.small_double_top)
#df = df.join(renko_df.up_trend_correction)
#df = df.join(renko_df.down_trend_correction)

#df = df.join(renko_peak_price_move)
#df = df.join(renko_fall_price_move)

In [ ]:
signals_df[signals_df.peak == True]

In [ ]:
signals_df = d.join(df)

trailing_stop = signals_df.exo.rolling(30).median().shift(1)

short_entry = (signals_df.peak == True)
short_exit = (signals_df.fall == True)
#short_exit = CrossDown(temp_df.exo, trailing_stop)

direction = -1
pl, inposition = backtester.backtest(signals_df, short_entry, short_exit, direction)
equity, stats = backtester.stats(pl, inposition)

equity.plot()

In [ ]:
signals_df.columns

In [ ]:
d.join(renko_df, on=renko_df['date'])

In [ ]:
d.join(df)

In [ ]:
t.count(1)